The data used here can be found [here](https://www.pxweb.bfs.admin.ch/Selection.aspx?px_language=fr&px_db=px-x-0702000000_104&px_tableid=px-x-0702000000_104\px-x-0702000000_104.px&px_type=PX).

We gathered the data from 2013-2015 (2016) is not available. However, the names of the municipalities are those from the year 2016 (or later). In ordrer to have some data for the right municipality names, we assume that the ratio cow/inhabitant doesn't change between the municipalities that merge together.

In [81]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import re

In [82]:
df = pd.read_csv('data_to_be_processed/cows.csv', encoding='latin_1', skiprows=2, delimiter=';')

In [83]:
df = df.drop(['Système d\'exploitation'], axis=1)
df.columns = ['year', 'commune', 'cows']
df.head()


,year,commune,cows
0,2013,- Suisse,1557474
1,2013,- Zürich,93233
2,2013,>> Bezirk Affoltern,9315
3,2013,......0001 Aeugst am Albis,622
4,2013,......0002 Affoltern am Albis,596


### Preprocessing

In [84]:
#remove districts
df = df.loc[(df.commune.str.contains('>>') == False)]
#remove cantons
df = df.loc[(df.commune.str.contains('- ') == False)]
#remove swiss total
df = df[df['commune'] != '- Suisse']
#rewrite commune name
df['id'], df['commune'] = df['commune'].str.split(' ', 1).str
regex = re.compile(r"\.{6}")
df['id'].replace(to_replace=regex, value='', inplace=True, regex=True)
df['id'] = df['id'].astype(int)


df = df[~df.commune.str.isnumeric()]

In [99]:
df[df.id == 5081]

,year,commune,cows,id
1766,2013,Sobrio,27,5081
4271,2014,Sobrio,32,5081
6776,2015,Sobrio,25,5081


In [85]:
f = pd.read_csv("data_to_be_processed/fusion_mappings.csv")
f = f[f.date.str.contains('2016')]
f.head()
df[df['id'] == 310]
f.shape

(54, 5)

In [86]:
f[f['post_name'] == 'Rapperswil (BE)']

,pre_id,pre_name,post_id,post_name,date
700,532,Bangerten,310,Rapperswil (BE),01.01.2016
783,310,Rapperswil (BE),310,Rapperswil (BE),01.01.2016


In [87]:
fu = pd.merge(f, df, left_on='post_name', right_on='commune', how='right')
fu['commune'] = fu['pre_name'].fillna(fu['commune'])

fu[fu['id'] == 310]

,pre_id,pre_name,post_id,post_name,date,year,commune,cows,id
6,532.0,Bangerten,310.0,Rapperswil (BE),01.01.2016,2013,Bangerten,1716,310
7,310.0,Rapperswil (BE),310.0,Rapperswil (BE),01.01.2016,2013,Rapperswil (BE),1716,310
8,532.0,Bangerten,310.0,Rapperswil (BE),01.01.2016,2014,Bangerten,1679,310
9,310.0,Rapperswil (BE),310.0,Rapperswil (BE),01.01.2016,2014,Rapperswil (BE),1679,310
10,532.0,Bangerten,310.0,Rapperswil (BE),01.01.2016,2015,Bangerten,1695,310
11,310.0,Rapperswil (BE),310.0,Rapperswil (BE),01.01.2016,2015,Rapperswil (BE),1695,310


In [88]:
fu = pd.merge(f, df, left_on='post_name', right_on='commune', how='right')
fu[fu.id.duplicated()]
fu['commune'] = fu['pre_name'].fillna(fu['commune'])
fu['compute_ratio_id'] = fu['id']
fu['id'] = fu['pre_id'].fillna(fu['id'])
fu['id'] = fu['id'].astype(int)
fu['compute_ratio_id'] = fu['compute_ratio_id'].astype(int)
fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
fu.head()

,year,commune,cows,id,compute_ratio_id
0,2013,Kyburg,1573,175,296
1,2013,Illnau-Effretikon,1573,174,296
2,2014,Kyburg,1590,175,296
3,2014,Illnau-Effretikon,1590,174,296
4,2015,Kyburg,1675,175,296


In [90]:
fu.shape[0]/3

2329.0

In [10]:
# tmp = df[df['year'] == 2015]
# # tmp = tmp.drop('year', axis=1)

# fu = pd.merge(f, df, left_on='pre_name', right_on='commune', how='outer')
# fu['commune'] = fu['post_name'].fillna(fu['commune'])
# fu['id'] = fu['post_id'].fillna(fu['id'])
# fu['id'] = fu['id'].astype(int)
# fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
# fu = fu.groupby(['id', 'commune', 'sex', 'nationality', 'age'], as_index=False).sum()

In [11]:
# def create_2016_data(data):
# regex = re.compile(r"\d\d\.\d\d\.")
# f['date'].replace(to_replace=regex, value='', inplace=True, regex=True)
# fu = pd.merge(f, tmp, left_on='post_name', right_on='commune', how='right')
# # fu['commune'] = fu['post_name'].fillna(fu['commune'])
# fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
# fu = fu.groupby(['commune', 'id', 'nationality', 'age'], as_index=False).sum()
#     return fu

In [12]:
# fu[fu.date.astype(float) < fu.year] # commune = pre_name, id = pr_id

# #final: compute_ratio_id, id, commune, cows, year
# #compute_ratio_id = id of municipality of current name (column 'id')
# #id, commune : id of municipality adjusted by the years (pre_id if mun has merged ( date > year), else id)
# #year = column year
# fu['compute_ratio_id'] = fu['id']
# #check if equality is in the right direction
# fu['id'] = fu.apply((lambda x: x.pre_id if (float(x.date) > x.year) else x.id ), axis=1 )
# fu['commune'] = fu.apply((lambda x: x.pre_name if (float(x.date) > x.year) else x.commune ), axis=1 )

In [91]:
fu.shape

(6987, 5)

In [14]:
# fu = fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1)

In [92]:
# fu['id'] = fu['id'].astype(int)
for year in range (2013, 2017):    
    #filter out by year
    if(year == 2016):
        tmp = fu[fu['year'] == 2015]
        tmp = tmp.drop('year', axis=1)
    else:
        tmp = fu[fu['year'] == year]
        tmp = tmp.drop('year', axis=1)
    tmp.to_csv('municipalities/%d/data_cows.csv' % year, index=False)
tmp.head()

,commune,cows,id,compute_ratio_id
4,Kyburg,1675,175,296
5,Illnau-Effretikon,1675,174,296
10,Bangerten,1695,532,310
11,Rapperswil (BE),1695,310,310
18,Niederösch,1016,417,405


In [93]:
tmp.shape
tmp[tmp.isnull().any(1)]

,commune,cows,id,compute_ratio_id


In [94]:
mun = pd.read_csv('municipalities/2016/data_commune.csv')

In [95]:
mun.shape

(2324, 9)

In [96]:
foo = pd.merge(tmp, mun, left_on='id', right_on='id', how='outer')


In [97]:
len(foo.id.unique())

2324

In [98]:
foo[foo.id.duplicated()]

,commune_x,cows,id,compute_ratio_id,commune_y,total_inhabitants,percentage_18,percentage_40,percentage_65,percentage_100,percentage_men,percentage_swiss
45,Sobrio,25,5081,5081,Sobrio,77,0.16,0.13,0.43,0.29,0.49,0.94
48,Vergeletto,0,5132,5132,Vergeletto,64,0.00,0.16,0.38,0.47,0.52,0.91
50,Gresso,0,5109,5109,Gresso,38,0.13,0.08,0.39,0.39,0.58,1.00
52,Mosogno,2,5119,5119,Mosogno,52,0.12,0.12,0.52,0.25,0.52,0.92
54,Isorno,0,5137,5137,Isorno,302,0.13,0.17,0.46,0.25,0.52,0.91


In [100]:
foo[foo.id == 5081]

,commune_x,cows,id,compute_ratio_id,commune_y,total_inhabitants,percentage_18,percentage_40,percentage_65,percentage_100,percentage_men,percentage_swiss
44,Sobrio,496,5081,5072,Sobrio,77,0.16,0.13,0.43,0.29,0.49,0.94
45,Sobrio,25,5081,5081,Sobrio,77,0.16,0.13,0.43,0.29,0.49,0.94
